In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jun 13 18:59:45 2021

@author: leodefaria
"""

## Importação de bibliotecas

In [ ]:
# from urllib.request import urlopen
# from urllib.error import HTTPError
# from urllib.error import URLError
# import ssl
import pandas as pd
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Importação de dados

In [ ]:
# Vacinados

# base original: DADOS ABERTOS - Registros de Vacinação COVID19 - UF: DF
# https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-06-12/uf%3DDF/part-00000-e7b5a3a9-bb01-40f5-aa63-d3bd2cfe2507.c000.csv
df_vacina = pd.read_csv('/Users/leodefaria/Documents/IDP/dados/part-00000-e7b5a3a9-bb01-40f5-aa63-d3bd2cfe2507.c000.csv', sep=';')

In [ ]:
# População

# Base Original: IBGE - Dados dos Municípios
# https://cidades.ibge.gov.br/brasil/df/brasilia/panorama
# Populacao = 3055149

In [ ]:
# População por faixas

# Base Original: IBGE - Projeção Municípios
# https://www.ibge.gov.br/apps/populacao/projecao/index.html
# df_IBGE = pd.read_csv('/Users/leodefaria/Documents/IDP/dados/DF - Dados.csv')
# df_IBGE = pd.read_csv('/Users/leodefaria/Documents/IDP/dados/DF - Fx_Idade.csv')
df_IBGE_F = pd.read_csv('/Users/leodefaria/Documents/IDP/dados/DF - Fx_IdadeM.csv', sep=';')
df_IBGE_M = pd.read_csv('/Users/leodefaria/Documents/IDP/dados/DF - Fx_IdadeH.csv', sep=';')


## Ajustes VACINA

In [ ]:
# apagando colunas desnecessárias para análise

del df_vacina['document_id']
del df_vacina['paciente_id']
del df_vacina['paciente_datanascimento']
del df_vacina['paciente_racacor_codigo']
del df_vacina['paciente_endereco_coibgemunicipio']
del df_vacina['paciente_endereco_copais']
del df_vacina['paciente_endereco_cep']
del df_vacina['paciente_nacionalidade_enumnacionalidade']
del df_vacina['estabelecimento_valor']
del df_vacina['estabelecimento_razaosocial']
del df_vacina['estabelecimento_municipio_codigo']
del df_vacina['estabelecimento_municipio_nome']
del df_vacina['estabelecimento_uf']
del df_vacina['vacina_grupoatendimento_codigo']
del df_vacina['vacina_grupoatendimento_nome']
del df_vacina['vacina_lote']
del df_vacina['vacina_fabricante_nome'] # apesar do HIPE não é neste campo que se encontra o fabricante
del df_vacina['vacina_fabricante_referencia']
del df_vacina['id_sistema_origem']
del df_vacina['sistema_origem']
del df_vacina['data_importacao_rnds']
del df_vacina['vacina_codigo']
del df_vacina['vacina_categoria_codigo']

In [ ]:
# Ajustando nomes colunas

df_vacina.rename(columns={  'paciente_idade':'idade'
                          , 'paciente_enumsexobiologico':'sexo'
                          , 'paciente_racacor_valor':'cor'
                          , 'paciente_endereco_nmmunicipio':'municipio'
                          , 'paciente_endereco_nmpais':'pais'
                          , 'paciente_endereco_uf':'uf'
                          , 'estalecimento_nofantasia':'local_vacinacao'
                          , 'vacina_categoria_nome':'vacina_categoria'
                          , 'vacina_dataaplicacao':'dt_aplicacao'
                          , 'vacina_descricao_dose':'dose'}, inplace=True)


In [ ]:
#df_vacina.describe()
df_vacina.head()

## Ajuste IBGE

In [ ]:
# Separaçao dos dados de 2020 e 2021

# Feminino
df_IBGE_F.drop(df_IBGE_F.columns[1:11], inplace=True, axis=1)
df_IBGE_F.drop(df_IBGE_F.columns[3:], inplace=True, axis=1)
df_IBGE_F.head()

# Masculino
df_IBGE_M.drop(df_IBGE_M.columns[1:11], inplace=True, axis=1)
df_IBGE_M.drop(df_IBGE_M.columns[3:], inplace=True, axis=1)
df_IBGE_M.head()

In [ ]:
campos=labels=['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90+']


df_vacina['Faixa/Ano'] = pd.cut(x=df_vacina['idade'], bins=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,100],labels=campos)

df_vacina.head()

In [ ]:

df_vacina_grupo = df_vacina.groupby(['Faixa/Ano']).agg('count')
df_vacina_grupo.head()

## Gráficos

In [ ]:
# Plot a distribuicao da idade dos vacinados

# sns.distplot(df_vacina.idade[df_vacina.sexo == 'F'], bins = np.linspace(1,70,35))
# sns.distplot(df_vacina.idade[df_vacina.sexo == 'M'], bins = np.linspace(1,70,35))
# sns.histplot(data=df_vacina, x='idade', hue='sexo', stat='count', bins=np.linspace(1,70,35), legend=True)


plt.figure(figsize=(12,12))

sns.violinplot(  data = df_vacina
               , x ='idade'
               , y ='sexo'
               , hue ='dose'
               , split = True
              )


plt.title('Distribuição da idade dos vacinados por sexo')
#plt.legend(['Feminino','Masculino'])
plt.xlabel('Idade')
plt.ylabel('Dist. por Sexo')
plt.show()

In [ ]:
# Teste valores: Aparentemente a 2dose está maior 

df_resumo = df_vacina.groupby(['dose','sexo']).agg('count')
df_resumo.head()